In [1]:
'''
#Train a simple deep CNN on the CIFAR10 small images dataset.
It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

In [2]:
#unzip train and test images
!unzip '/content/drive/My Drive/deeper_systems/train.rotfaces.zip'

Archive:  /content/drive/My Drive/deeper_systems/train.rotfaces.zip
replace train.truth.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train/0-10049200_1891-09-16_1958.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [3]:
!unzip '/content/test.rotfaces.zip'

Archive:  /content/test.rotfaces.zip
replace test/90-10184590_1979-06-16_2006.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [6]:
#model parameters
batch_size = 32
num_classes = 4
epochs = 6
data_augmentation = False
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'


In [23]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import cv2
import numpy as np

#get train and validation images

df = pd.read_csv('/content/train.truth.csv')
le = LabelEncoder()
df['encoded_label'] = le.fit_transform(df.label)

train, test = train_test_split(df, test_size=0.05)

img_list = []
for filename in train.fn:
  img_list.append(cv2.imread('train/{filename}'.format(filename=filename)))

x_train = np.array(img_list)
x_train.shape

(46451, 64, 64, 3)

In [10]:
img_list = []
for filename in test.fn:
  img_list.append(cv2.imread('train/{filename}'.format(filename=filename)))

x_test = np.array(img_list)
x_test.shape

(2445, 64, 64, 3)

In [11]:
#get ground truth

#le = LabelEncoder()
#y_train = le.fit_transform(train.label)
#y_test = LabelEncoder().fit_transform(test.label)

y_train = keras.utils.to_categorical(train.encoded_label, 4)
y_test = keras.utils.to_categorical(test.encoded_label, 4)

y_test.shape

(2445, 4)

In [12]:
#build model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


In [13]:
# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255



In [14]:
#train model
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)



Not using data augmentation.
Epoch 1/6
1452/1452 [==============================] - 40s 27ms/step - loss: 0.4886 - accuracy: 0.8082 - val_loss: 0.2280 - val_accuracy: 0.9194
Epoch 2/6
1452/1452 [==============================] - 39s 27ms/step - loss: 0.2046 - accuracy: 0.9279 - val_loss: 0.1329 - val_accuracy: 0.9538
Epoch 3/6
1452/1452 [==============================] - 39s 27ms/step - loss: 0.1354 - accuracy: 0.9534 - val_loss: 0.1014 - val_accuracy: 0.9652
Epoch 4/6
1452/1452 [==============================] - 39s 27ms/step - loss: 0.1048 - accuracy: 0.9642 - val_loss: 0.0969 - val_accuracy: 0.9693
Epoch 5/6
1452/1452 [==============================] - 39s 27ms/step - loss: 0.0883 - accuracy: 0.9693 - val_loss: 0.0762 - val_accuracy: 0.9730
Epoch 6/6
1452/1452 [==============================] - 39s 27ms/step - loss: 0.0795 - accuracy: 0.9735 - val_loss: 0.0740 - val_accuracy: 0.9751


In [15]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)



Saved trained model at /content/saved_models/keras_cifar10_trained_model.h5 


In [16]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

77/77 [==============================] - 1s 8ms/step - loss: 0.0740 - accuracy: 0.9751
Test loss: 0.07400631159543991
Test accuracy: 0.9750511050224304


In [17]:
import os

directory = os.fsencode('/content/test')

img_list = []
for file in os.listdir(directory):
     filename = os.fsdecode(file)
     img_list.append(cv2.imread('/content/test/{filename}'.format(filename=filename)))

arr_test = np.array(img_list)
arr_test.shape

(5361, 64, 64, 3)

In [18]:
model.predict(arr_test)

array([[0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [29]:
for pred in model.predict(arr_test):
  print(le.inverse_transform([np.argmax(pred, axis=None, out=None)]))

Streaming output truncated to the last 5000 lines.
['rotated_right']
['rotated_left']
['upside_down']
['upright']
['upside_down']
['upside_down']
['upright']
['upside_down']
['rotated_right']
['rotated_left']
['rotated_right']
['rotated_left']
['upside_down']
['upside_down']
['rotated_left']
['rotated_right']
['rotated_right']
['upright']
['rotated_left']
['rotated_left']
['upside_down']
['upside_down']
['upside_down']
['rotated_right']
['rotated_left']
['upside_down']
['rotated_right']
['upright']
['rotated_left']
['upside_down']
['upside_down']
['upside_down']
['rotated_left']
['upside_down']
['upright']
['rotated_right']
['rotated_left']
['upside_down']
['upright']
['rotated_left']
['rotated_left']
['rotated_right']
['upright']
['rotated_right']
['rotated_right']
['upside_down']
['rotated_left']
['upside_down']
['rotated_right']
['upright']
['rotated_left']
['rotated_right']
['rotated_left']
['upside_down']
['upright']
['upside_down']
['upright']
['rotated_left']
['rotated_left']
['